# Sankey Graph of Officers and Entity Jurisdictions

This Jupyter Notebook analyzes entity data to understand the relationships between officer origin countries and entity registration jurisdictions. It loads and cleans the data, then filters for the top 5 entity jurisdictions and the top 10 officer origin countries based on entity flow counts. Finally, the notebook visualizes these flows using a Plotly Sankey diagram and calculates the proportion of entities from each top origin country going to each top jurisdiction.

In [1]:
# import packages to create sankey graphs

import plotly.graph_objects as go
import pandas as pd
import plotly.express as px

In [2]:
# load data

df = pd.read_csv("ICIJ_Merged.csv")
df.head()

/var/folders/5x/0sqjbpy13p3gzjlddpxqjtfw0000gn/T/ipykernel_16286/3436424098.py:3: DtypeWarning: Columns (4,5,6,7,8,10,11,14,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,37,39,40,41,42,43,45,46,47,48,49,50,51,52,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("ICIJ_Merged.csv")


,node_id_start,node_id_end,rel_type,link,status,start_date,end_date,sourceid,address_start,name_start,...,internal_id_end,incorporation_date_end,inactivation_date_end,struck_off_date_end,dorm_date_end,status_end,service_provider_end,ibcruc_end,type_end,closed_date_end
0,10002580,14106952,registered_address,registered address,NaN,NaN,NaN,Panama Papers,STEMBRIDGE TRUST (IRELAND) LIMITED 5 THE COURT...,FUSION TRADING LIMITED,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10004460,14101133,registered_address,registered address,NaN,NaN,NaN,Panama Papers,MF CORPORATE (UK) LIMITED 520 S. 7TH STREET SU...,MF CORPORATE (UK) LIMITED,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10023813,14105100,registered_address,registered address,NaN,NaN,NaN,Panama Papers,REIG INVESTMENTS LLP INVISION HOUSE WILBURY WA...,REIG INVESTMENTS LLP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10023840,14100712,registered_address,registered address,NaN,NaN,NaN,Panama Papers,MARCUSSI INTERNATIONAL LLP INVISION HOUSE WILB...,MARCUSSI INTERNATIONAL LLP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10010428,14093957,registered_address,registered address,NaN,NaN,NaN,Panama Papers,COLLYER BRISTOW LLP SOLICITORS 4 BEDFORD ROW L...,PULSAR HOLDINGS CORPORATION,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Determine the top 5 jurisdictions based on the count of entities.
# It filters the dataframe for rows where 'node_type_end' is 'entity',
# then counts the occurrences of each 'jurisdiction_description_end',
# selects the top 5 most frequent/popular, and converts their names to a list.
top_jurisdictions = (
    df[df['node_type_end'] == 'entity']['jurisdiction_description_end']
    .value_counts()
    .nlargest(5)
    .index
    .tolist()
)

# Create new dataframe which includes rows where:
# 1. The 'node_type_start' is 'officer'.
# 2. The 'node_type_end' is 'entity'.
# 3. The 'jurisdiction_description_end' is one of the 'top_jurisdictions' identified above.
df_filtered = df[
    (df['node_type_start'] == 'officer') & 
    (df['node_type_end'] == 'entity') &
    (df['jurisdiction_description_end'].isin(top_jurisdictions))
]

In [4]:
print(df_filtered['jurisdiction_description_end'].value_counts())

jurisdiction_description_end
Malta                     565267
British Virgin Islands    391519
Bermuda                   165947
Barbados                  142731
Bahamas                    52283
Name: count, dtype: int64


In [5]:
print(top_jurisdictions)

['Malta', 'British Virgin Islands', 'Bahamas', 'Bermuda', 'Barbados']


**Note:** we will use the top 5 jurisdictions that are found by the following code

In [6]:
# Further filter df_filtered to ensure only top jurisdictions are considered (this step might be redundant if df_filtered is already correctly constructed).
# Then, count the occurrences of each jurisdiction in 'jurisdiction_description_end'.
juris_counts = (
    df_filtered[df_filtered['jurisdiction_description_end'].isin(top_jurisdictions)]
    ['jurisdiction_description_end']
    .value_counts()
    .reset_index() # Convert the Series to a DataFrame and make the index (jurisdictions) a column.
)

# Rename the columns of the new dataframe for clarity.
juris_counts.columns = ['jurisdiction', 'count']

print(juris_counts) 

             jurisdiction   count
0                   Malta  565267
1  British Virgin Islands  391519
2                 Bermuda  165947
3                Barbados  142731
4                 Bahamas   52283


In [7]:
# Fill NaN values in 'countries_start' with an empty string and ensure the column is of string type
df_filtered['countries_start'] = df_filtered['countries_start'].fillna("").astype(str)
# Split the 'countries_start' string by ';' to handle multiple countries listed in a single entry
df_filtered['countries_start'] = df_filtered['countries_start'].str.split(';')
# Transform each item in the lists (from the split) into a separate row, duplicating other column values
df_filtered = df_filtered.explode('countries_start')

# Remove leading/trailing whitespace from the 'countries_start' entries
df_filtered['countries_start'] = df_filtered['countries_start'].str.strip()

# Filter out rows where 'countries_start' is an empty string after stripping
df_filtered = df_filtered[df_filtered['countries_start'] != ""]

# Filter out rows where 'countries_start' is "not identified" (case-insensitive) or an empty string
df_filtered = df_filtered[
    (df_filtered['countries_start'].str.lower() != "not identified") &
    (df_filtered['countries_start'] != "")
]
# Count the occurrences of each country in 'countries_start' and convert the result to a dataframe
country_counts = df_filtered['countries_start'].value_counts().reset_index()
# Rename the columns of the new dataframe for clarity
country_counts.columns = ['country', 'count']

print(country_counts.head(10))

/var/folders/5x/0sqjbpy13p3gzjlddpxqjtfw0000gn/T/ipykernel_16286/404896383.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['countries_start'] = df_filtered['countries_start'].fillna("").astype(str)
/var/folders/5x/0sqjbpy13p3gzjlddpxqjtfw0000gn/T/ipykernel_16286/404896383.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['countries_start'] = df_filtered['countries_start'].str.split(';')


                  country   count
0                   Malta  387148
1          United Kingdom   97694
2           United States   75592
3  British Virgin Islands   74252
4                 Bermuda   66634
5               Hong Kong   59287
6               Singapore   57816
7               Indonesia   48366
8                   China   45750
9                Thailand   41195


In [8]:
# Group data by officer origin country ('countries_start') and entity jurisdiction ('jurisdiction_description_end'),
# count the number of occurrences for each combination, and reset the index to create a dataframe
# The resulting 'flow_counts' df will have columns: 'countries_start', 'jurisdiction_description_end', and 'count'
flow_counts = df_filtered.groupby(['countries_start', 'jurisdiction_description_end']).size().reset_index(name='count')

# Create a list of the top 5 jurisdictions to focus on which were previously identified
top_jurisdictions = ['Malta', 'British Virgin Islands', 'Bahamas', 'Bermuda', 'Barbados']

# Filter to include only rows where 'jurisdiction_description_end' is one of the specified 'top_jurisdictions'
flow_counts = flow_counts[flow_counts['jurisdiction_description_end'].isin(top_jurisdictions)]

# Determine the top 10 officer origin countries ('countries_start') based on the sum of 'count'
# (i.e., total flow from each country to the selected top jurisdictions)
top_origins = (
    flow_counts.groupby('countries_start')['count']
    .sum()
    .nlargest(10)
    .index
    .tolist()
)

# Create a unique list of labels for the sankey graph nodes
# This list includes the top origin countries and the top jurisdictions
# Jurisdictions are added only if they are not already present in the top_origins list (to avoid duplicates)
labels = list(top_origins) + [j for j in top_jurisdictions if j not in top_origins]

# Create a dictionary mapping each label (country or jurisdiction name) to a unique numerical index
label_to_index = {label: i for i, label in enumerate(labels)}

# Add a 'source' column to 'flow_counts' by mapping the 'countries_start' values to their corresponding numerical indices using 'label_to_index'
flow_counts['source'] = flow_counts['countries_start'].map(label_to_index)

# Add a 'target' column to 'flow_counts' by mapping the 'jurisdiction_description_end' values to their corresponding numerical indices using 'label_to_index'
flow_counts['target'] = flow_counts['jurisdiction_description_end'].map(label_to_index)


In [9]:
print(flow_counts['jurisdiction_description_end'].value_counts())

jurisdiction_description_end
British Virgin Islands    235
Malta                     197
Bahamas                   155
Bermuda                   154
Barbados                   14
Name: count, dtype: int64


In [10]:
# Define a list of qualitative colors from Plotly Express for styling
colors = px.colors.qualitative.Plotly

# Create a dictionary to map each top origin country to a unique color
# Colors are cycled if the number of countries exceeds the number of available colors
origin_color_map = {country: colors[i % len(colors)] for i, country in enumerate(top_origins)}

# Filter the flow_counts DataFrame to include only flows:
# Originating from one of the 'top_origins' countries and destined for one of the 'top_jurisdictions'
flow_counts = flow_counts[
    flow_counts['countries_start'].isin(top_origins) &
    flow_counts['jurisdiction_description_end'].isin(top_jurisdictions)
]

# Add a 'link_color' column to flow_counts, assigning a color to each flow (link)
# based on its origin country using the 'origin_color_map'
flow_counts['link_color'] = flow_counts['countries_start'].map(origin_color_map)

# Create a combined list of labels for sankey graph nodes: top origin countries followed by top jurisdictions
# This list defines all unique nodes in the plot.
labels = list(top_origins) + top_jurisdictions

# Create a dictionary mapping each label (country or jurisdiction name) to a unique numerical index
# Sankey diagrams require numerical indices for sources and targets.
label_to_index = {label: i for i, label in enumerate(labels)}

# Assign colors to each node:
# Origin countries get their specific color from 'origin_color_map'
# Other nodes (jurisdictions) are colored 'lightgray'
node_colors = [origin_color_map.get(label, "lightgray") for label in labels]

# Create the sankey graph 
fig = go.Figure(data=[go.Sankey(
    # Define node properties
    node=dict(
        pad=30,  # Padding between nodes
        thickness=20,  # Thickness of the nodes
        line=dict(color="black", width=0.5),  # Border line for nodes
        label=labels,  # Labels for the nodes
        color=node_colors  # Colors for the nodes
    ),
    # Define link properties
    link=dict(
        source=flow_counts['countries_start'].map(label_to_index),  # Map origin countries to numerical source indices
        target=flow_counts['jurisdiction_description_end'].map(label_to_index),  # Map destination jurisdictions to numerical target indices
        value=flow_counts['count'],  # Thickness of the links based on flow count
        color=flow_counts['link_color']  # Color of the links based on origin country
    )
)])

fig.update_layout(
    title_text="Top Officer Countries → Top 5 Entity Jurisdictions",  
    font=dict(size=12), 
    height=600, 
    width=1000  
)

fig.show()

### Creating summary table of sankey graph

In [11]:
# Group data by origin country and destination jurisdiction, then sum the counts for each group
summary_table = (
    flow_counts.groupby(['countries_start', 'jurisdiction_description_end'])['count']
    .sum()
    .reset_index()
)

# calculate the proportion of flow for each origin country to each jurisdiction
# for each origin country, calculate the count for each jurisdiction as a fraction of the total count for that origin
summary_table['proportion'] = summary_table.groupby('countries_start')['count'].transform(
    lambda x: x / x.sum()
)

# Convert the proportion to a percentage and round to two decimal places
summary_table['proportion'] = (summary_table['proportion'] * 100).round(2)

# Rename columns for better readability
summary_table = summary_table.rename(columns={
    'countries_start': 'Origin Country',
    'jurisdiction_description_end': 'Destination Jurisdiction',
    'count': 'Flow Count',
    'proportion': 'Proportion (%)'
})

# Display the resulting summary table
summary_table.head

<bound method NDFrame.head of             Origin Country Destination Jurisdiction  Flow Count  \
0                  Bermuda                  Bahamas          12   
1                  Bermuda                 Barbados          10   
2                  Bermuda                  Bermuda       65116   
3                  Bermuda   British Virgin Islands        1073   
4                  Bermuda                    Malta         423   
5   British Virgin Islands                  Bahamas         940   
6   British Virgin Islands                 Barbados           2   
7   British Virgin Islands                  Bermuda        1361   
8   British Virgin Islands   British Virgin Islands       68932   
9   British Virgin Islands                    Malta        3017   
10                   China                  Bahamas          91   
11                   China                  Bermuda        4545   
12                   China   British Virgin Islands       38766   
13                   China      